In [1]:
import numpy as np
import math as m

In [2]:

import opensim as osim
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import json
import os

### Import Transformed Markers

In [3]:
markers_df = pd.read_csv('model_update/tps_warping_results/markers_transformed.csv', index_col=0)

In [4]:
markers_df[markers_df['body']=='pelvis']

,body,location
name,,
ASIS_l,pelvis,[-7.18673236e-05 2.39908737e-05 -1.09069255e-01]
ASIS_r,pelvis,[ 7.18673236e-05 -2.39908737e-05 1.09069255e-01]
PSIS_l,pelvis,[-0.12422751 -0.00100463 -0.05171399]
PSIS_r,pelvis,[-0.12231541 0.00105886 0.05155155]
pelvis_origin,pelvis,[8.17044429e-09 2.61787435e-09 3.18476136e-09]
ilium_r,pelvis,[-0.07435212 0.04666253 0.09570157]
ilium_l,pelvis,[-0.07203954 0.0393926 -0.10045404]
isch_spine_r,pelvis,[-0.07183451 -0.08881641 0.05315165]
isch_spine_l,pelvis,[-0.07022732 -0.08781739 -0.04701425]


### Import Transformed Muscle Paths

In [5]:
mscles_df = pd.read_csv('model_update/tps_warping_results/muscles_transformed.csv', index_col=0)
mscles_df.head()

,body,location
name,,
addbrev_r-P1,pelvis,[ 0.00090114 -0.09667376 0.01693799]
addlong_r-P1,pelvis,[ 0.00818057 -0.08844186 0.01996903]
addmagDist_r-P1,pelvis,[-0.02191218 -0.13458107 0.03836024]
addmagIsch_r-P1,pelvis,[-0.03598057 -0.14105504 0.04186994]
addmagMid_r-P1,pelvis,[-0.01001184 -0.12573645 0.02820166]


### Import Transformed Wrapping Surfaces

In [6]:
wrp_df = pd.read_csv('model_update/tps_warping_results/wrp_transformed.csv', index_col=1)
wrp_df.head()

,body,translation,rotation,length,radius
name,,,,,
Gmax1_at_pelvis_r,pelvis,"[-0.0406, -0.1113, 0.0669]","[-0.56813, -0.30577, 0.0186]",0.1700,0.0321
Gmax2_at_pelvis_r,pelvis,"[-0.0486, -0.0963, 0.0755]","[-0.75703, -0.33133, 0.0176]",0.1104,0.0324
Gmax3_at_pelvis_r,pelvis,"[-0.0502, -0.1022, 0.0757]","[-0.13198, -0.18094, 0.00784]",0.1122,0.0327
Gmax1_at_pelvis_l,pelvis,"[-0.0356, -0.1097, -0.0609]","[0.51118, 0.26581, 0.01389]",0.1658,0.0323
Gmax2_at_pelvis_l,pelvis,"[-0.0431, -0.0947, -0.0686]","[0.70918, 0.29079, 0.01311]",0.1090,0.0330


### Parse current scaled model

In [7]:
tree=ET.parse("scaled_model_P01.osim")
root = tree.getroot()

### Update reference geometry

In [8]:
ref_geom_dic = {'body': [], 'Mesh name' : [], 'Mesh file' : []}

ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\r_pelvis.stl')
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_2"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\l_pelvis.stl')
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_3"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\sacrum.stl')

ref_geom_dic['body'].append("femur_r"); ref_geom_dic['Mesh name'].append("femur_r_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\femur_r.stl')
ref_geom_dic['body'].append("femur_l"); ref_geom_dic['Mesh name'].append("femur_l_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\femur_l.stl')

ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\tibia_r.stl')
ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_2"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\fibula_r.stl')

ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\tibia_l.stl')
ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_2"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\fibula_l.stl')

ref_geom_dic['body'].append("patella_r"); ref_geom_dic['Mesh name'].append("patella_r_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\patella_r.stl')
ref_geom_dic['body'].append("patella_l"); ref_geom_dic['Mesh name'].append("patella_l_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\patella_l.stl')

In [9]:
ref_geom_df = pd.DataFrame(ref_geom_dic).set_index('Mesh name')

In [10]:
for Mesh in root.iter('Mesh'):
    point = Mesh.attrib['name']
    if point in ref_geom_df.index:
        new_text = ref_geom_df.loc[point, 'Mesh file']
        # print(point, new_text)
        Mesh.find('mesh_file').text = new_text
        Mesh.find('scale_factors').text = '1 1 1'
    else: pass

### Update muscle paths

In [11]:
for PathPoint in root.iter('PathPoint'):
    point = PathPoint.attrib['name']
    if point in mscles_df.index:
        location = mscles_df.loc[point, 'location']
        new_text = location[1:-1]
        # print(point, new_text)
        PathPoint.find('location').text = new_text
    else: pass

### Update locations of muscle wrapping surfaces

In [12]:
for WrapCylinder in root.iter('WrapCylinder'):
    point = WrapCylinder.attrib['name']
    if point in wrp_df.index:
        translation = wrp_df.loc[point, 'translation']
        new_text = translation[1:-1]
        # print(point, new_text)
        WrapCylinder.find('translation').text = new_text
    else: pass

### Update Translation of joint centres

In [13]:
femur_r_center_in_pelvis = markers_df.loc['femur_r_center_in_pelvis', 'location']
femur_l_center_in_pelvis = markers_df.loc['femur_l_center_in_pelvis', 'location']
knee_l_center_in_femur = markers_df.loc['knee_l_center_in_femur_l', 'location']
knee_r_center_in_femur = markers_df.loc['knee_r_center_in_femur_r', 'location']
patella_l_center_in_femur = markers_df.loc['patella_l_in_femur_l', 'location']
patella_r_center_in_femur = markers_df.loc['patella_r_in_femur_r', 'location']

talus_l_center_in_tibia = markers_df.loc['talus_l_center_in_tibia', 'location']
talus_r_center_in_tibia = markers_df.loc['talus_r_center_in_tibia', 'location']

In [14]:
for joint in root.iter('CustomJoint'):
    
    name = joint.attrib['name']
    #ground_pelvis
    if joint.attrib['name'] ==  "ground_pelvis":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_r_center_in_pelvis[1:-1]
                print('old r hip translation', frame.find('translation').text)
                frame.find('translation').text = new_text
                print('new r hip translation', frame.find('translation').text)
            else: pass

    if joint.attrib['name'] ==  "hip_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_r_center_in_pelvis[1:-1]
                print('old r hip translation', frame.find('translation').text)
                frame.find('translation').text = new_text
                print('new r hip translation', frame.find('translation').text)
            else: pass
    
    elif joint.attrib['name'] ==  "hip_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_l_center_in_pelvis[1:-1]
                frame.find('translation').text = new_text
            else: pass
            
    elif joint.attrib['name'] ==  "walker_knee_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_r_offset":
                new_text = knee_r_center_in_femur[1:-1]
                print('old r walker knee translation in femur',frame.find('translation').text)
                frame.find('translation').text = new_text
                print('new r walker knee translation in femur',frame.find('translation').text)
            #if frame.attrib['name'] == "tibia_r_offset":
            #    new_text = f"{walker_knee_r_tibia_r_offset[0]}, {walker_knee_r_tibia_r_offset[1]}, {walker_knee_r_tibia_r_offset[2]}"
            #    print ('old r tibia translation', frame.find('translation').text)
            #    frame.find('translation').text = new_text
            #    print ('new r tibia translation', frame.find('translation').text)
            else: pass    
    
    elif joint.attrib['name'] ==  "walker_knee_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_l_offset":
                new_text = knee_l_center_in_femur[1:-1]
                frame.find('translation').text = new_text
    #        if frame.attrib['name'] == "tibia_l_offset":
    #            new_text = f"{walker_knee_l_tibia_l_offset[0]}, {walker_knee_l_tibia_l_offset[1]}, {walker_knee_l_tibia_l_offset[2]}"
    #            frame.find('translation').text = new_text
            else: pass

    elif joint.attrib['name'] ==  "patellofemoral_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_r_offset":
               new_text = knee_r_center_in_femur[1:-1]
               print('patellofemoral r old', frame.find('translation').text)
               frame.find('translation').text = new_text
               print('patellofemoral r new', frame.find('translation').text)
           else: pass
    
    elif joint.attrib['name'] ==  "patellofemoral_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_l_offset":
               new_text = knee_l_center_in_femur[1:-1]
               frame.find('translation').text = new_text
           else: pass

# for joint in root.iter('PinJoint'):
    
#     name = joint.attrib['name']

#     if joint.attrib['name'] ==  "ankle_r":
#        for frame in joint.iter('PhysicalOffsetFrame'):
#            if frame.attrib['name'] == "tibia_r_offset":
#                new_text = talus_r_center_in_tibia [1:-1]
#                print('ankle r old', frame.find('translation').text)
#                frame.find('translation').text = new_text
#                print('ankle r new', frame.find('translation').text)
#            else: pass
    
#     elif joint.attrib['name'] ==  "ankle_l":
#        for frame in joint.iter('PhysicalOffsetFrame'):
#            if frame.attrib['name'] == "tibia_l_offset":
#                new_text = talus_l_center_in_tibia [1:-1]
#                frame.find('translation').text = new_text
#            else: pass
#     else: pass

old r hip translation 0 0 0
new r hip translation -0.03280726 -0.08525383  0.08542795
old r hip translation -0.052491341063049979 -0.072375734846084103 0.072064130544659197
new r hip translation -0.03280726 -0.08525383  0.08542795
old r walker knee translation in femur -0.0073979812802869782 -0.37306309556314898 -0.0025147649593064507
new r walker knee translation in femur -5.50670620e-17 -3.79492327e-01  3.55271368e-18
patellofemoral r old -0.0073979812802869782 -0.37306309556314898 -0.0025147649593064507
patellofemoral r new -5.50670620e-17 -3.79492327e-01  3.55271368e-18
ankle r old -0.01048876039206271 -0.41955041568250845 0
ankle r new -0.00498055 -0.39347991 -0.0023595 


### Update Markers for Joint Positions in Parent

In [ ]:
['torso_origin_in_pelvis','femur_l_center_in_pelvis', 'femur_r_center_in_pelvis',  'knee_l_center_in_femur_l', 'patella_l_in_femur_l', 'knee_r_center_in_femur_r', 'patella_r_in_femur_r'] # , 'ankle_l_tibia_l_offset', 'ankle_r_tibia_r_offset' ankle_l_tibia_l_offset = talus_l_center_in_tibia; ankle_r_tibia_r_offset = talus_r_center_in_tibia

['torso_origin_in_pelvis',
 'femur_l_center_in_pelvis',
 'femur_r_center_in_pelvis',
 'knee_l_center_in_femur_l',
 'patella_l_in_femur_l',
 'knee_r_center_in_femur_r',
 'patella_r_in_femur_r',
 'ankle_l_tibia_l_offset',
 'ankle_r_tibia_r_offset']

In [ ]:
m_names = ['torso_origin_in_pelvis','femur_l_center_in_pelvis', 'femur_r_center_in_pelvis',  'knee_l_center_in_femur_l', 'patella_l_in_femur_l', 'knee_r_center_in_femur_r', 'patella_r_in_femur_r']

for marker in root.iter('Marker'):
    name = marker.attrib['name']
    if name in m_names:
        old_text = marker.find('location').text
        new_text = markers_df.loc[name, 'location']
        marker.find('location').text = new_text[1:-1]


### Export transformed model

In [15]:
tree.write('p01_tps_transformed.osim')